## Explicit Euler vs. Implicit-in-velocity Euler
Explicit Euler and implicit-in-velcoity Euler are both used to update the state of a system. However, they work a little bit differently. Explicit Euler updates the state based on the the current state of the system as you can see from the equation:
$$
y_{n+1} = y_n + \Delta t \cdot f(t_n, y_n)
$$
Where:
- $y_{n+1}$ is the state at the next time step
- $y_n$ is the current state at the current time step
- $\Delta t \cdot$ is the time step
- $f(t_n, y_n)$ is the function that describes the system's dynamics at the current time step

While, implicit-in-velocity Euler updates the velocity implicity and the positions explicitly. The equation for velocity is:
$$
v_{t+h} = v_t + h * a(v_{t+h})
$$
Where:
- $v_{t+h}$ is the velocity at the next time step (this is unknown and has to be solved for)
- $v_t$ is the current velocity
- $h$ is the time step
- $a(v_{t+h})$ is the acceleration at the next time step, which depends on the velocity at the next time step

Since $a(v_{t+h})$ is a function of $v_{t+h}$ this forms a non-linear equation. In order to solve for $v_{t+h}$ Mujoco uses a **first-order Taylor Expansion** around $v_t$ since we cannot evaluate $a(v_{t+h})$ without the next velocity. This takes the form of: 
$$
a(v_{t+h}) \approx a(v_t) + \frac{\partial v}{\partial a(v)} \cdot (v_{t+h} - v_t)
$$
Where:
- $a(v_{t+h})$ repesents the acceleration at time $t+h$ and is a function of velocity at the next time step
- $a(v_t)$ is the acceleration at the current time and is a function of the current velocity
- $\frac{\partial v}{\partial a(v)} \cdot$ is the derivative of velocity with respect to acceleration, which is used to approximate the acceleration at the next time step
- $(v_{t+h} - v_t)$ is the change in velocity over the next time step

The derivative is given by:
$$
\frac{\partial v}{\partial a(v)} = M^{-1} D
$$
Where: 
- $M^{-1}$ is the inverse of the mass matrix for the system. $M$ represents the distribution of mass in the system and how it relates to forces acting on the objects
- $D$ is the damping matrix that encodes how the forces change with velocities, excluding constraint forces. 

This simplifies to the following equation to give us the velocity update: 
$$
v_{t+h} = v_t + h \hat{M}^{-1} M a(v_t)
$$
With:
$$
\hat{M} \equiv M - hD
$$

The position is then updated using $v_{t+h}$:
$$
q_{t+h} = q_t + h * v_{t+h}
$$

The main similarities between these two algorithms are that they both predict the state of the system at the next time step based on the current conditions, and they are both first-order integration methods meaning they use the first derivative or the rate of change of the system's state to predict the next state. The main difference is that one is explicit and the other is implicit. Explicit means that it computes the next state of the system using values that have already been computed, while implicit uses values that depend on the unknown future state. Another difference is the explicit Euler is stable for smaller time steps, but can become unstable if the time step is too large. The implicit-in-velocity Euler tends to be more stable for stiff systems since it computes the velocity from the next time step. The explicit Euler is also relatively simple to implement, as we did in class, while the implicit-in-velocity Euler requires solving a non-linear equation for the velocity update which is much more difficult. Overall, both algorithms update the state of the system, explicit Euler is easier to implement, but less stable, and the implicit-in-velovity Euler is more complex, but more stable especially for systems with damping, springs, and/or stiff systems.



In [61]:
import mujoco
import mediapy as media
import numpy as np

# Load the model from the XML file
model = mujoco.MjModel.from_xml_path("trampoline.xml")  # Ensure the file path is correct
data = mujoco.MjData(model)

# Define simulation parameters
duration = 5.0  # Duration of the simulation in seconds
framerate = 60  # Frames per second (video framerate)

# Reset the simulation state and time
mujoco.mj_resetData(model, data)

# Capture frames during the simulation
frames = []
with mujoco.Renderer(model) as renderer:
    while data.time < duration:
        mujoco.mj_step(model, data)  # Step forward in time

        # Capture frames at the specified framerate
        if len(frames) < data.time * framerate:
            renderer.update_scene(data)  # Update the scene
            pixels = renderer.render()   # Render the current frame
            frames.append(pixels)        # Append to the list of frames

# Display the video
media.show_video(frames, fps=framerate)


This browser does not support the video tag.


## Chosen Example: Trampoline Simulation
Every simulation in Mujoco is based on the simple principle that force is equal to mass multiplied by acceleration. In practice, this gets very complicated very quickly, but the most simple explanation for how it all works is that forces are used to find accelerations which are used to find velocities at the next timestep which are used to update positions for the next timestep. In the trampoline example, we start with a square, flexible body which is supported by pins at each of its corners and a ball positioned some distance above the "trampoline". When the simulation starts, all motion is due to gravity which Mujoco considers to be a passive force meaning that it only depends on position and velocity. The contact force which occurs when the ball hits the trampoline depends on many more factors. The ball itself is a simple rigid body (meaning it does not deform) that has a mass, but the trampoline geometry has many more variables which influence its reaction namely the Young's modulus, Poisson's ratio, thickness, and damping coefficient. Because the trampoline is generated by creating a grid of points with flexible connections between them, the contact force is not as simple as it would be with a ball bouncing off of a rigid wall. 

When the ball hits the trampoline, Mujoco simultaneously calculates the forces on each point in the flexible grid and the ball using a system of equations which ensures that everything matches up correctly. Poisson's ratio is set to zero in this example, so it does not affect anything, but the Young's modulus of the trampoline designates how much the material deforms in reaction to forces on it. This allows the trampoline to act like a spring in that the more it is deformed the more force it will push back with. The thickness of the trampoline also has an impact on how much it will deform under force in the same way that a single peice of paper is much easier to bend than a whole ream. The damping coefficient controls the force which opposes motion meaning that a higher damping coefficient will cause more of the energy to be absorbed at impact as opposed to a lower coefficient which will result in more elastic collisions. Damping is also controlled in two seperate locations: there is one number set for the edges of the grid forming the trampoline which only affects the motion of the trampoline itself and another which governs the actal contact forces. The parameter solimp allows the user to govern how quickly forces increase with penetration of the flex body, and the parameter solref allows the user to set the stiffness of the material and the damping coefficient of contact. Ultimately, in this initial example, the lower Young's modulus and higher time constant in the solref parameter result in a trampoline that does not actually really bounce the ball at all. Therefore, we set out to modify this simulation into something more accurately representing a trampoline.

In [103]:
# Load the model from the XML file
model = mujoco.MjModel.from_xml_path("trampoline2.xml")
data = mujoco.MjData(model)

# Set initial velocity for the ball (assuming it's the first body)
data.qvel[0:3] = [0, 0, -10]  # Set linear velocity in x, y, z directions

# After loading the model
model.opt.gravity[2] = -9.81  # Reduce gravity (default is -9.81)

# Define simulation parameters
duration = 10.0  # Duration of the simulation in seconds
framerate = 60  # Frames per second (video framerate)

# Reset the simulation state and time
mujoco.mj_resetData(model, data)

# Capture frames during the simulation
frames = []
with mujoco.Renderer(model) as renderer:
    while data.time < duration:
        mujoco.mj_step(model, data)  # Step forward in time

        # Capture frames at the specified framerate
        if len(frames) < data.time * framerate:
            renderer.update_scene(data)  # Update the scene
            pixels = renderer.render()   # Render the current frame
            frames.append(pixels)        # Append to the list of frames

# Display the video
media.show_video(frames, fps=framerate)

This browser does not support the video tag.


In [150]:
import mujoco
import mediapy as media
import numpy as np

# Load the model from the XML file
model = mujoco.MjModel.from_xml_path("tramphumanoid.xml")  # Ensure the file path is correct
data = mujoco.MjData(model)

model.opt.gravity[2] = -12  # Reduce gravity (default is -9.81)
# Define simulation parameters
duration = 8.0  # Duration of the simulation in seconds
framerate = 60  # Frames per second (video framerate)

# Reset the simulation state and time
mujoco.mj_resetData(model, data)

# Capture frames during the simulation
frames = []
with mujoco.Renderer(model) as renderer:
    while data.time < duration:
        mujoco.mj_step(model, data)  # Step forward in time

        # Capture frames at the specified framerate
        if len(frames) < data.time * framerate:
            renderer.update_scene(data)  # Update the scene
            pixels = renderer.render()   # Render the current frame
            frames.append(pixels)        # Append to the list of frames

# Display the video
media.show_video(frames, fps=framerate)

This browser does not support the video tag.


## Extensions and changes made to Trampoline Simulation
We started with wanting to make the ball bounce higher on the trampoline since the original simulation looks like a ball landing in a tarp. To do this we changed the flexcomp type from a grid to a disc giving the trampoline a round design. Next we decided to pin down the trampoline around the edges. We had to understand how the layout worked and find the correct pin ids. This did add some extra bounce since the force was more distributed. To go even further, we increased Young's Modulus which makes the material stiffer resulting in less deformation of the trampoline and also made the material thicker. We then changed the contact properties of the trampoline, specifically solref and solimp. Solref is the Solver Reference which controls how contact forces are resolved. We made this more precise which improved the trampoline's responsiveness and bounce. The solimp is Solver Impulse which helps control damping and collisions. We increased this allowing the trampoline to dissipate less energy. We replaced the ball with a humanoid figure by using capsule and sphere geoms and connected them together using multiple joints. To do this we had to look at the original humanoid xml. We increased the damping and stiffness of the humanoid to allow it to not be so much of a ragdoll. We also played with the gravity of the simulation. 

The first difference from the original simulation can be seen immediately when the simulation begins. The material making up the trampoline does not sag downwards under the force of gravity like in the original example. This is due to both the increased thickness and Young's modulus. A thicker material takes more force to deform as explained above, and the increased modulus means it takes more force to get the same amount of deformation seen in the original example. These parameters work together to make the trampoline stiffer and more life-like. Increasing the solimp parameter means that forces grow quicker as the falling object penetrates the trampoline's surface. This gives the trampoline more bounce instead of catching the object as seen in the original example. Decreasing the time constant in the solref parameter from 0.01s to 0.001s means that contact forces are restored 10 times faster than they were in the original simulation. This again results in stiffer behavior from the trampoline which translates to more bounciness. Overall, we were able to fine tune all of the parameters of the simulation to transform the originally terrible, not bouncy trampoline into one that much more closely mimics the behavior one would expect from a real trampoline.